# Juliaでラムダ計算
このNotebookは、`succ`、`pred`等のラムダ計算をJuliaで実際に動かしてみるというものです。  
この実験にあたって、デフォルトで部分適用可能な関数を吐き出す`@lambda`マクロを作りました。  はじめにラムダ計算について知っていることを簡単にまとめ、次に`@lambda`の文法と、それらを機能させるために必要な関数を解説し、その後ラムダ計算を用いて**チャーチ数**の算術を実際に書いてみます。  
ラムダ計算については、[このブログ](http://clz.hatenablog.com/entry/2017/02/25/233100)を参考にしました。  

# [ラムダ計算](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%83%A0%E3%83%80%E8%A8%88%E7%AE%97)とは
**ラムダ計算**は次のような文法で定義されたプログラミング言語の一種であると言うことができます。
    
```
<expr> ::= <identifier>
<expr> ::= (λ <identifier>. <expr>)
<expr> ::= (<expr> <expr>)
```
ラムダ計算の文法は、*変数*と*関数*そして*関数適用*が定義されていて、それらはすべてラムダ式である、ということになります。 例えば2つの引数を足し合わせる計算`add`は、`+`という記号を認めるならば、

$$
\begin{eqnarray}
add = \lambda x.\lambda y.\,y+x
\end{eqnarray}
$$

と定義することができます。  
ここで、`add　3`という記号について考えてみます。  
`add`は上の通りに定義した変数であり、`3`をリテラルとして認めることにすると、`add 3`は2つの式からなる関数適用であると言えます。元々`add` は2つの引数を取る計算だったはずですが、関数適用であるならば`add 3`はラムダ式でなければなりません。そこで`add 3`は次のように計算します。

$$
\begin{eqnarray}
add 3 & = &(\lambda x.\lambda y.\,y+x)\;3\\
& = &\lambda y.\,y+3
\end{eqnarray} 
$$

すなわち、`add 3`は`y` を引数に取って`y`より`3`大きな数字を返す関数である、と解釈します。
この`x`を`3`に置き換える操作を、ラムダ計算の世界では**β簡約**いいます。Juliaプログラミングにおいてβ簡約のような操作をするには、必要な引数のうちの一部のみを引数にして関数を呼び出す**部分適用**ができるように関数を定義する必要があります。

### マクロ `@lambda`
Juliaで自動で部分適用ができる関数を定義するために、引数の個数だけネストしたクロージャを返す関数を出力するマクロ`@lambda`と、複数の引数をネストされたクロージャに一つずつ適用する演算子`<|`を定義します。  
`@lambda`は`@lambda arg1, arg2, ... , argN, = <expr>` という文法で使います。  

```julia
例
[in]   add = @lambda x, y, z, = x+y+z   # add = @lambda x, = @lambda y, = @lambda z, = x+y+z
[out] add
(x -> begin
    y -> begin
        z -> begin
            z + y + x
        end
    end
end)

[in]  @lambda g, f, x, = g(f(x))
[out]
(g -> begin
    f -> begin
        x -> begin
            g(f(x))
        end
    end
end)
```
`<|`は`<Function> <| (arg1, arg2, ..., argN)`という書き方をします。  

```julia
例
[in]   add <| (1, 2, 3)...
[out] 6

[in]   add <| (1, 2)
[out] (::getfield(Main, Symbol("#<<numer>>#"))) 
(z -> begin
    z + y + z
end)

```
`@apply`と`@calc`は次に説明する**チャーチ数**から実際の値を求めるためのマクロです。端的に言えば`@apply f x`はSchemeの`(f x)`と同じ意味を持ちます。以下はソースコードです。


In [1]:
macro lambda(tpl::Union{Symbol, Expr}...)
    e = tpl[1]
    expr = :($(e.args[2]))
    for arg in reverse(e.args[1].args)
        expr = :($arg -> $expr)
    end
    esc(expr)
end

function <|(f::Function, args...)
    
    if length(args) < 2
        f(args[1])
    else
        f(args[1]) <| args[2:end]...
    end
    
end

#β-reduction
macro apply(lambda::Union{Expr, Symbol} ...)
    f = lambda[1]
    x = lambda[2:end]
    
    expr = :(<|($f, $(x...)))
    expr = :($f <| $(x...))
    esc(expr)
end

macro calc(lambda::Union{Expr, Symbol})
    args = (g::Function -> () -> g()+1, ()->0)
    expr = :($lambda <| $(args...))
    esc(:($expr()))
    end

@calc (macro with 1 method)

### チャーチ数と算術
チャーチ数とは、関数適用の繰り返しで自然数を定義するものです。具体的には、2つの引数`f` `x`を受け取り`f`を$n$回繰り返し呼び出すラムダ式として書くことができます。<br>
例えば、$n=0$, $n=1$, $n=2$ならば

$$
\begin{eqnarray}
zero & = & \lambda f.\lambda x.\,x\\ 
one  & = & \lambda f.\lambda x.\,f\;x\\
two & = & \lambda f.\lambda x.\,f\;(f\;x)
\end{eqnarray}
$$

となります。  
ところで、[ペアノの公理](https://ja.wikipedia.org/wiki/%E3%83%9A%E3%82%A2%E3%83%8E%E3%81%AE%E5%85%AC%E7%90%86#%E5%AE%9A%E7%BE%A9)に従って自然数を構成するには、チャーチ数$n$を引数にとってその**後者**を返す$successor$（以下$succ$）という関数を構成しなければなりません。$succ$はラムダ式で次のように定義できます。

$$
\begin{eqnarray}
succ & = & \lambda n.\lambda f.\lambda x.\,f\;(n\;f\;x)
\end{eqnarray}
$$

試しに$one = \lambda f.\lambda x.\,f\;x$の**後者**を計算してみます。

$$
\begin{eqnarray}
succ\;one & = & (\lambda n.\lambda f.\lambda x.\,f\;(n\;f\;x))\;(\lambda f.\lambda x.\,f\;x)\\
& = & \lambda f.\lambda x.\,f\;((\lambda f.\lambda x.\,f\;x)\;f\;x)\\
& = & \lambda f.\lambda x.\,f\;(f\;x)\\
& = & two
\end{eqnarray}
$$

無事に$one$の**後者**$two$が出てきました。  
自然数には他にも**加算**$add$、**乗算**$mul$、**冪乗**$pow$があり、チャーチ数に対しても定義できます。それらはすべて[ハイパー演算子](https://ja.wikipedia.org/wiki/%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E6%BC%94%E7%AE%97%E5%AD%90)として実装されます。簡潔に説明すると、$add$は$succ$の繰り返し適用、$mul$は$add$の繰り返し適用,$pow$は$mul$の繰り返し適用、...　というように、演算子を一つ階数の低い演算子の繰り返し適用として再帰的に定義するものです。  
また、$successor$の逆写像$predecessor$（以下$pred$）も構成できます。複雑な操作なので説明は省略します。

### Juliaでラムダ計算

ここまで見てきたラムダ計算に従ってチャーチ数と演算をJuliaで実装してみます。  
以下がコードです。ラムダ式による表現は[こちら](http://clz.hatenablog.com/entry/2017/02/25/233100)を参考にしてください。

In [2]:
Zero = @lambda f::Function, x::Function, = x
One = @lambda f::Function, x::Function, = @apply f x


succ = @lambda n::Function, f::Function, x::Function, = @apply f (@apply n f x)
pred = @lambda n::Function, f::Function, x::Function, = @apply n (@lambda y::Function, z::Function, = @apply z @apply y f) (@lambda y::Function, =x) (@lambda y::Function, =y)

add = @lambda n::Function, m::Function, = @apply m succ n
mul = @lambda n::Function, m::Function, = @apply m (@apply add n) Zero
pow = @lambda n::Function, m::Function, = @apply m (@apply mul n) One

#45 (generic function with 1 method)

では実際に計算してみます。  
マクロ`@calc`は、関数オブジェクトであるチャーチ数に対して`f = g -> () -> g()+1`, `x = () -> 0`という引数を渡し、返ってきた関数を`call`します。  
意味としては、ほぼ以下と等価なプログラムです。  
```julia
let f = g -> () -> g()+1, x = () -> 0
    let church = f(f(x))
        church()  # => 2
    end
end
```

まずは$succ$が正しく動くか試します。  
`1`の2つ**後者**`3`を求めます。

In [3]:
Three = @apply succ @apply succ One
@calc Three

3

`3`の１つ前者`2`は`pred Three`で求めます。

In [4]:
Two = @apply pred Three
@calc Two

2

次は$3^4$を計算してみます。

In [5]:
let Three = (@apply succ @apply succ One),　Four = (@apply succ @apply succ @apply succ One)
    @calc @apply pow Three Four 
end

81

無事に動くことが確認できました。